# Downloads Publication Information for PANGO Lineages from the CORD-19 Data Set
**[Work in progress]**

This notebook text-mines [PANGO lineage](https://cov-lineages.org/) mentions in the titles and abstracts of publications and preprints from the CORD-19 data set. Note, the text-mined results may contain false positive!

Data sources: [PANGO Lineage Designations](https://github.com/cov-lineages/pango-designation), 
[CORD-19](https://allenai.org/data/cord-19)

References:

Rambaut A, et al., A dynamic nomenclature proposal for SARS-CoV-2 lineages to assist genomic epidemiology(2020) Nature Microbiology [doi:10.1038/s41564-020-0770-5](https://doi.org/10.1038/s41564-020-0770-5).

Lucy Lu Wang, et al., CORD-19: The COVID-19 Open Research Dataset (2020) [arXiv:2004.10706v4](https://arxiv.org/abs/2004.10706).

Author: Peter Rose (pwrose@ucsd.edu)

In [489]:
import os
import pandas as pd
import io
import dateutil
import re
from pathlib import Path
import nltk
import json, requests
from urllib.request import urlopen
from xml.etree.ElementTree import parse
import urllib
import time
import numpy as np

In [490]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [573]:
NEO4J_IMPORT = "/Users/lyt/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-a1516f46-b63a-46dd-b67a-1fb59d6c5d05/import"#Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/lyt/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-a1516f46-b63a-46dd-b67a-1fb59d6c5d05/import


## Get PANGO lineages

In [574]:
pango = pd.read_csv(NEO4J_IMPORT + "/00b-PANGOLineage.csv", dtype=str)

In [575]:
pango.sample(5)

,lineage,description,alias,predecessor,l0,l1,l2,l3,levels
528,B.1.1.424,"Russia lineage, contains some formally B.1.1.1...",NaN,NaN,B.1.1.424,B.1.1,B.1,B,4
924,U.1,"Alias of B.1.177.60.1, England",B.1.177.60.1,B.1.177.60,U.1,U,NaN,NaN,2
422,AZ.6,"Alias of B.1.1.318.6, Canada lineage, from pan...",B.1.1.318.6,B.1.1.318,AZ.6,AZ,NaN,NaN,2
945,B.1.177.80,Scandinavian lineage,NaN,NaN,B.1.177.80,B.1.177,B.1,B,4
920,B.1.177.57,UK lineage,NaN,NaN,B.1.177.57,B.1.177,B.1,B,4


In [576]:
lineages = pango['lineage'].unique()

In [578]:
pattern1 = re.compile(' [A-Z]{1,2}[.]\d+ ', re.IGNORECASE)
pattern2 = re.compile(' [A-Z]{1,2}[.]\d+[.]\d+ ', re.IGNORECASE)
pattern3 = re.compile(' [A-Z]{1,2}[.]\d+[.]\d+[.]+\d+ ', re.IGNORECASE)

# add WHO lineage
who_lineage = [' Alpha ', ' Beta ', ' Gamma ', ' Epsilon ',' Zeta ', ' Eta ', ' Theta  ',\
               ' Iota ', ' Kappa ', ' Lambda ', ' Mu ']
pattern4 = re.compile("|".join(who_lineage), re.IGNORECASE)

In [579]:
# add who to lineages
lineages = np.append(lineages, who_lineage)

## Get CORD-19 Metadata

In [ ]:
CACHE = Path(NEO4J_IMPORT +'/cache/cord19/2022-03-31/metadata.csv')

In [ ]:
metadata = pd.read_csv(CACHE, dtype='str')

In [ ]:
metadata.fillna('', inplace=True)
#convert datetime column to just date
metadata['year'] = metadata['publish_time'].apply(lambda d: d[:4] if len(d) > 4 else '')
metadata['date'] = metadata['publish_time'].apply(lambda d: dateutil.parser.parse(d) if len(d) > 0 else '')

In [ ]:
print("Total number of papers", metadata.shape[0])

## Extract a list of PANGO lineages

Remove special characters to simply parsing for lineages in parenthesis, comma-separated lists, etc.

In [ ]:
metadata['title'] = metadata['title'].replace('[()/,]', ' ', regex=True)
metadata['abstract'] = metadata['abstract'].replace('[()/,]', ' ', regex=True)

Match PANGO patterns and check agains list of known lineages.

In [556]:
pattern1 = re.compile(' [A-Z]{1,2}[.]\d+ ', re.IGNORECASE)
pattern2 = re.compile(' [A-Z]{1,2}[.]\d+[.]\d+ ', re.IGNORECASE)
pattern3 = re.compile(' [A-Z]{1,2}[.]\d+[.]\d+[.]+\d+ ', re.IGNORECASE)

# add WHO lineage
who_lineage = [' Alpha ', ' Beta ', ' Gamma ', ' Epsilon ',' Zeta ', ' Eta ', ' Theta  ',\
               ' Iota ', ' Kappa ', ' Lambda ', ' Mu ']
pattern4 = re.compile("|".join(who_lineage), re.IGNORECASE)

In [ ]:
# add who to lineages
lineages = np.append(lineages, who_lineage)

In [46]:
def get_lineages(row):
    text = ' ' + row.title + ' ' + row.abstract + ' '
    lin = pattern1.findall(text) + pattern2.findall(text) + pattern3.findall(text)
    u_lin = set()
    
    
    for l in lin:
        l = l.strip()
        # check if lineage is valid (e.g., not a withdrawn lineage or false positive)
        if l in lineages:
            u_lin.add(l)
            
    return ";".join(u_lin)

### Run on whole dataset

In [32]:
metadata['lineages'] = metadata.apply(get_lineages, axis=1)

Keep only papers that map to PANGO lineages

In [33]:
hits = metadata[metadata['lineages'].str.len() > 0].copy()

### Assign CURIEs from [Identifiers.org](https://identifiers.org)

In [34]:
hits['doi'] = hits['doi'].apply(lambda x: 'doi:' + x if len(x) > 0 else '')
hits['pubmed_id'] = hits['pubmed_id'].apply(lambda x: 'pubmed:' + x if len(x) > 0 else '')
hits['pmcid'] = hits['pmcid'].apply(lambda x: 'pmc:' + x if len(x) > 0 else '')
hits['arxiv_id'] = hits['arxiv_id'].apply(lambda x: 'arxiv:' + x if len(x) > 0 else '')

In [35]:
#hits.sort_values(by=['publish_time'], ascending=False, inplace=True)

In [36]:
print("Number of matches", hits.shape[0])

Number of matches 4419


In [37]:
def create_id(row):
    """Creates a unique id using the most commonly available id in priority order"""
    if row.doi != '':
        return row.doi
    elif row.pubmed_id != '':
        return row.pubmed_id
    elif row.pmcid != '':
        return row.pmcid
    elif row.arxiv_id != '':
        return row.arxiv_id
    elif row.url != '':
        return row.url
    else:
        # TODO deal with WHO papers here?
        return ''

In [38]:
hits['id'] = hits.apply(create_id, axis=1)

WHO documents seem to be copies of articles that are already present in the dataset and will be ignored for now.

In [40]:
hits.query('id != ""', inplace=True)

In [41]:
print("Total number of matches", hits.shape[0])

Total number of matches 3200


In [44]:
hits.to_csv(NEO4J_IMPORT + "01h-CORDLineages.csv", index=False)

## Fulltext Regrex


In [580]:
# get articles ids for specific lineage
def get_ids(lineage):
    url = requests.get(f'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=(%22{lineage}%22%20AND%20(%22SARS-CoV-2%22%20OR%20%22COVID-19%22)%20AND%20(%22lineage%22%20OR%20%22lineages%22%20OR%20%22strain%22%20OR%20%22strains%22%20OR%20%22variants%22%20OR%20%22variants%22))%20AND%20(FIRST_PDATE:%5b2020-01-01%20)%20AND%20HAS_FT:y%20AND%20%20sort_date:y&resultType=idlist&pageSize=1000&format=json&cursorMark=*')
    text = url.text
    results = json.loads(text)['resultList']['result']
    ids = list(map(lambda x: x['fullTextIdList']['fullTextId'][0], results))
    return ids

In [581]:
# download articles in XML and return body paragraph
def download_article(article_id):
    url = f'https://www.ebi.ac.uk/europepmc/webservices/rest/{article_id}/fullTextXML'
    xmldoc = parse(urlopen(url))
    
    # get full text
    root = xmldoc.getroot()
    text = root.findall('.//p')

    # put body paragraphs together
    ptext = ""
    for p in text:
        ptext += ''.join([x for x in p.itertext()]) + '.\n' + '\n'
    return ptext

In [582]:
# get lineage for full texts
def get_full_lineage(ptext):
    # tokenize texts into sentences
    p_sentence = nltk.tokenize.sent_tokenize(ptext)
    
    # record lineages
    linset = set()
    pair = []
    for s in p_sentence:
        s1 = re.subn('[()/,]', ' ', s)[0] # remove special chars
        lin = set(pattern1.findall(s1) + pattern2.findall(s1) + pattern3.findall(s1) + pattern4.findall(s1))

        if lin: 
            for l in lin:
                # valid lineage and not recorded
                l = l.strip()
                l = l.capitalize()
                if (l in lineages) and (l not in linset): 
                    linset.add(l)
                    pair.append([l, s])
                else: continue

    
    """
    ptext = re.subn('[()/,]', ' ', ptext)[0] # remove special chars
    lin = pattern1.findall(ptext) + pattern2.findall(ptext) + pattern3.findall(ptext)
    lin_set = set(lin)
    
    record = []
    if lin_set:
        for l in lin_set:
            
            sen = re.search(r"\.?([^\.]*{}[^\.]*)".format(l), ptext).group()
            record.append([l, sen])
    """
    return pair

In [502]:
# wrap up function take lineage as input and output dataframe
def extract_full(lineage):
    ids = get_ids(lineage)
    full_regrex = []
    if ids:
        for i in ids:
            try: 
                path = Path(f'{lineage}/{i}.txt')

                # if file not exist, get body text and save to file
                if not path.is_file():
                    body_text = download_article(i)
                    path.parent.mkdir(parents=True, exist_ok=True)
                    with path.open("w", encoding ="utf-8") as f:
                        f.write(body_text)
                        f.close()
                else: # otherwise retrieve text
                    body_text = path.read_text()

                record = get_full_lineage(body_text) # get lineages
                [x.append(i) for x in record] # attach article id to lineage record
                full_regrex.append(pd.DataFrame(record))
            except urllib.error.HTTPError as exc:
                time.sleep(10) # wait 10 seconds and then make http request again
                continue
        df_fulltext = pd.concat(full_regrex)
        return df_fulltext
    else: return None

    

#### test on B.1.1.7


In [231]:
lineage = 'B.1.1.7'
ids = get_ids(lineage)

full_regrex = []
for i in ids:
    try: 
        path = Path(f'{lineage}/{i}.txt')
        
        # if file not exist, get body text and save to file
        if not path.is_file():
            body_text = download_article(i)
            path.parent.mkdir(parents=True, exist_ok=True)
            with path.open("w", encoding ="utf-8") as f:
                f.write(body_text)
                f.close()
        else: # otherwise retrieve text
            body_text = path.read_text()
        
        
        record = get_full_lineage(body_text) # get lineages
        [x.append(i) for x in record] # attach article id to lineage record
        full_regrex.append(pd.DataFrame(record))
    except urllib.error.HTTPError as exc:
        time.sleep(10) # wait 10 seconds and then make http request again
        continue

fulltext_lineage = pd.concat(full_regrex)

In [233]:
fulltext_lineage.to_csv('B_1_1_7.csv',index=False, header = ['lineage', 'string contains lineage', 'ID'])

#### test on P.1

In [234]:
lineage = 'P.1'
ids = get_ids(lineage)

full_regrex = []
for i in ids:
    try: 
        path = Path(f'{lineage}/{i}.txt')
        
        # if file not exist, get body text and save to file
        if not path.is_file():
            body_text = download_article(i)
            path.parent.mkdir(parents=True, exist_ok=True)
            with path.open("w", encoding ="utf-8") as f:
                f.write(body_text)
                f.close()
        else: # otherwise retrieve text
            body_text = path.read_text()
        
        
        record = get_full_lineage(body_text) # get lineages
        [x.append(i) for x in record] # attach article id to lineage record
        full_regrex.append(pd.DataFrame(record))
    except urllib.error.HTTPError as exc:
        time.sleep(10) # wait 10 seconds and then make http request again
        continue

fulltext_lineage = pd.concat(full_regrex)

In [253]:
fulltext_lineage.to_csv('P_1.csv',index=False, header = ['lineage', 'string contains lineage', 'ID'])

### manual check possible false postives
#### B.1.1.7

In [276]:
b117 = pd.read_csv('B_1_1_7.csv')

In [277]:
b117 = b117.drop(['Unnamed: 0'],axis = 1)
b117.columns = ['lineage', 'string contains lineage', 'ID']

In [346]:
# 1. same sentence with many lineages are counted as positive, we remove those 
b117_sub = b117[~ b117.duplicated('string contains lineage',keep=False)]

In [349]:
# 2. extract articles with only one lineage, which are possibly FP
IDs = b117_sub.groupby('ID').lineage.count().loc[lambda p : p == 1].index

In [350]:
b117_sub = b117_sub.set_index('ID').loc[IDs]

In [369]:
# manual check
#print(b117_sub['string contains lineage'].str.cat(sep = '\n '))

#### P.1

In [370]:
p1 = pd.read_csv("P_1.csv")
p1_sub = p1[~ p1.duplicated('string contains lineage',keep=False)]
IDs_1 = p1_sub.groupby('ID').lineage.count().loc[lambda p : p == 1].index

In [371]:
p1_sub = p1_sub.set_index('ID').loc[IDs_1]

In [379]:
p1_sub[p1_sub['string contains lineage'].str.contains('The nucleic acid')]

,lineage,string contains lineage
ID,,
PPR282435,R.9.4.1,The nucleic acid was converted to cDNA and amp...


## Start Generalization

In [583]:
# remove A B
lineages = np.delete(lineages, np.where(lineages == 'A'))
lineages = np.delete(lineages, np.where(lineages == 'B'))

In [584]:
import time

def iter_lineage(lineage):
    for l_ in lineage:
        print(f'start {l_}')
        '''if Path(f'{l_}/{l_}_df').is_file():
            found.add(l_)
            continue'''
        if l_ == 'B.1.1.7' or l_ == 'P.1':
            continue
        start = time.time()
        df = extract_full(l_)
        if df is None: 
            print(f'nothing for {l_}')
            nothing.add(l_)
        else:
            found.add(l_)
            df.to_csv(f'{l_}/{l_}_df',index=False, \
                                        header = ['lineage', 'string contains lineage', 'ID'])
            end = time.time()
            print(f'done with {l_}, time duration --- seconds --- {end - start} \n')
            

In [560]:
l_10 = lineages[:10]
l_10

array(['A.1', 'A.2', 'A.2.2', 'A.2.3', 'A.2.4', 'A.2.5', 'A.2.5.1',
       'A.2.5.2', 'A.2.5.3', 'A.3'], dtype=object)

In [561]:
nothing = set()
found = set()
iter_lineage(l_100)

start A.1
done with A.1, time duration --- seconds --- 259.9149000644684 

start A.2
done with A.2, time duration --- seconds --- 195.07675504684448 

start A.2.2
done with A.2.2, time duration --- seconds --- 72.12044787406921 

start A.2.3
done with A.2.3, time duration --- seconds --- 12.405742168426514 

start A.2.4
done with A.2.4, time duration --- seconds --- 1.5441129207611084 

start A.2.5
done with A.2.5, time duration --- seconds --- 104.74268698692322 

start A.2.5.1
done with A.2.5.1, time duration --- seconds --- 0.9085829257965088 

start A.2.5.2
done with A.2.5.2, time duration --- seconds --- 1.22562575340271 

start A.2.5.3
done with A.2.5.3, time duration --- seconds --- 0.8530848026275635 

start A.3
done with A.3, time duration --- seconds --- 216.7770800590515 



In [586]:
iter_lineage(['J.1'])

start J.1
done with J.1, time duration --- seconds --- 177.6583170890808 

